In [ ]:
#@title Project, Authentication and Big Query Client

#After creating the bucket "bucket-bdcc24-project1"
#And uploading there the .csv files

#Project
PROJECT_ID = 'bdcc24-project1' #@param {type: "string"}

#Authentication
from google.colab import auth
auth.authenticate_user()
#!gcloud config set project {PROJECT_ID}

#Big Query Client
import google.cloud.bigquery as bq
client = bq.Client(project=PROJECT_ID)


In [ ]:
#@title Create Dataset and Load Tables

#After creating the dataset "openimages"
#And creating the tables using the .csv files in the bucket

#Create Dataset
dataset = client.create_dataset('openimages', exists_ok=True)

#Load the Tables
import pandas as pd
dataset_id = PROJECT_ID+".openimages"

#Classes
classes_ref = dataset_id+".classes"
classes_df = client.query(f"SELECT * FROM `{classes_ref}`").to_dataframe()
classes_df = classes_df.rename(columns={"string_field_0": "Label", "string_field_1": "Description"})
classes_df = classes_df.drop(0)

#Image Labels
image_labels_ref = dataset_id+".image-labels"
image_labels_df = client.query(f"SELECT * FROM `{image_labels_ref}`").to_dataframe()
image_labels_df = image_labels_df.rename(columns={"string_field_0": "ImageId", "string_field_1": "Label"})
image_labels_df = image_labels_df.drop(0)

#Relations
relations_ref = dataset_id+".relations"
relations_df = client.query(f"SELECT * FROM `{relations_ref}`").to_dataframe()
relations_df = relations_df.rename(columns={"string_field_0": "ImageId", "string_field_1": "Label1", "string_field_2": "Relation", "string_field_3": "Label2"})
relations_df = relations_df.drop(0)

In [ ]:
#@title Load Tables in Big Query

#Classes
classes_ref2 = dataset_id+".classes2"
client.delete_table(classes_ref2, not_found_ok=True)
classes_table = bq.Table(classes_ref2)
classes_table.schema = (
        bq.SchemaField('Label',      'STRING'),
        bq.SchemaField('Description','STRING')
)
client.create_table(classes_table)
classes_load = client.load_table_from_dataframe(classes_df, classes_table)

#Image Labels
image_labels_ref2 = dataset_id+".image-labels2"
client.delete_table(image_labels_ref2, not_found_ok=True)
image_labels_table = bq.Table(image_labels_ref2)
image_labels_table.schema = (
    bq.SchemaField('ImageId', 'STRING'),
    bq.SchemaField('Label',   'STRING')
)
client.create_table(image_labels_table)
image_labels_load = client.load_table_from_dataframe(image_labels_df, image_labels_table)

#Relations
relations_ref2 = dataset_id+".relations2"
client.delete_table(relations_ref2, not_found_ok=True)
relations_table = bq.Table(relations_ref2)
relations_table.schema = (
    bq.SchemaField('ImageId',  'STRING'),
    bq.SchemaField('Label1',   'STRING'),
    bq.SchemaField('Relation', 'STRING'),
    bq.SchemaField('Label2',   'STRING'),
)
client.create_table(relations_table)
relations_load = client.load_table_from_dataframe(relations_df, relations_table)

In [ ]:
#@title Join Tables in Pandas

#Joined1
joined_df = pd.merge(classes_df, image_labels_df, on='Label')

#Joined2
classes_df = classes_df.rename(columns={'Label': 'Label1', 'Description': 'Description1'})
joined2_df = pd.merge(relations_df, classes_df, on='Label1')
classes_df = classes_df.rename(columns={'Label1': 'Label2', 'Description1': 'Description2'})
joined2_df = pd.merge(joined2_df, classes_df, on='Label2')
joined2_df['FinalRelation'] = joined2_df['Description1']+' '+joined2_df['Relation']+' '+joined2_df['Description2']

#Joined3
joined3_df = pd.merge(joined_df, joined2_df, on='ImageId')
joined3_df.drop(columns=['Label'], inplace=True)
joined3_df.rename(columns={'Description': 'Class'}, inplace=True)

In [ ]:
#@title Load Joined Tables in Big Query

#Joined1
joined_ref = dataset_id+".joined"
client.delete_table(joined_ref, not_found_ok=True)
joined_table = bq.Table(joined_ref)
joined_table.schema = (
    bq.SchemaField('ImageId',       'STRING'),
    bq.SchemaField('Label',         'STRING'),
    bq.SchemaField('Description',   'STRING')
)
client.create_table(joined_table)
joined_load = client.load_table_from_dataframe(joined_df, joined_table)

#Joined2
joined2_ref = dataset_id+".joined2"
client.delete_table(joined2_ref, not_found_ok=True)
joined2_table = bq.Table(joined2_ref)
joined2_table.schema = (
    bq.SchemaField('ImageId',       'STRING'),
    bq.SchemaField('Label1',        'STRING'),
    bq.SchemaField('Label2',        'STRING'),
    bq.SchemaField('Relation',      'STRING'),
    bq.SchemaField('Description1',  'STRING'),
    bq.SchemaField('Description2',  'STRING'),
    bq.SchemaField('FinalRelation', 'STRING')
)
client.create_table(joined2_table)
joined2_load = client.load_table_from_dataframe(joined2_df, joined2_table)

#Joined3
joined3_ref = dataset_id+".joined3"
client.delete_table(joined3_ref, not_found_ok=True)
joined3_table = bq.Table(joined3_ref)
joined3_table.schema = (
    bq.SchemaField('Class',         'STRING'),
    bq.SchemaField('ImageId',       'STRING'),
    bq.SchemaField('Label1',        'STRING'),
    bq.SchemaField('Label2',        'STRING'),
    bq.SchemaField('Relation',      'STRING'),
    bq.SchemaField('Description1',  'STRING'),
    bq.SchemaField('Description2',  'STRING'),
    bq.SchemaField('FinalRelation', 'STRING')
)
client.create_table(joined3_table)
joined3_load = client.load_table_from_dataframe(joined3_df, joined3_table)